In [1]:
import pandas as pd
import os
import json
import tarfile
import time

In [ ]:
# use this command to compress the uncompressed tarfile when the scrape is ready:
# xz -zk tarfile.tar

In [6]:
#tar = tarfile.open("../rawdata.tar.xz", "r:xz")
tar = tarfile.open("../databases/archives/2022_12_27_rawdata.tar.xz", "r:xz")

In [26]:
def create_nutrtion_df(data, name):
    transformed_data = [name] + [row[2] for row in data["data"]]
    columns = ["id"] + [row[0] for row in data["data"]]
    return pd.DataFrame([transformed_data], columns=columns)

In [33]:
start_time = time.time()
all_nutrition_df = []
for member in tar.getmembers():
    if member.name.find("nutrition.json") != -1:
        file_content = tar.extractfile(member.name).read().decode("utf-8")
        data = json.loads(file_content)
        all_nutrition_df.append(create_nutrtion_df(data, member.name.split("/")[-2]))

nutritions_df = pd.concat(all_nutrition_df, ignore_index=True)
end_time = time.time()
print(end_time - start_time)
print((end_time - start_time) / len(nutritions_df))

219.7322838306427
0.020729460738739876


In [34]:
nutritions_df

,id,Energia (kJ / kcal),Zsír (g),Telített zsírsavak (g),Szénhidrát (g),Cukrok (g),Rost (g),Fehérje (g),Só (g),Riboflavin (mg),...,Ebből koleszterin,A vitamin (µg),Tiamin (mg),Niacin (mg),Pantoténsav (mg),Klorid (mg),Cink (mg),Mangán (mg),Fluor (µg),Króm (µg)
0,14914,1199/285,8.8000,4.8000,42.0000,2.3000,0.000000,8.3000,1.4000,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3963016,1091.830000/260.770000,7.670000,3.540000,45.350000,2.690000,3.840000,0.670000,3.840000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4001800,1270/303,14.0000,7.6000,39.0000,17.0000,NaN,4.1000,0.7800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4001797,1122/268,13.0000,7.0000,34.0000,14.0000,NaN,3.7000,0.7000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3988465,1755.000000/420.000000,23.400000,15.000000,42.400000,12.600000,NaN,8.100000,0.840000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10595,4117341,2030/601,23,9,67,33,NaN,"5,6","0,66",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10596,4117392,129/31,"0,4","0,1","6,1","0,2",NaN,"0,7","0,9",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10597,4117335,2079/625,28,11,61,30,NaN,"5,5","1,23",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10598,4117305,1835/437,18,"7,2","63,6",28,NaN,"5,1","0,3",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
nutritions_df.to_csv("../databases/nutritions_raw.csv", escapechar="\\")

In [3]:
def create_df(data: dict):
    return pd.DataFrame([data.values()], columns=list(data.keys()))

In [10]:
start_time = time.time()
all_product_df = []
all_variant_df = []
all_prices_df = []
for member in tar.getmembers():
    if member.name.find("product.json") != -1:
        file_content = tar.extractfile(member.name).read().decode("utf-8")
        data = json.loads(file_content)

        price_data = data["selectedVariant"]["price"]
        price_data["id"] = data["selectedVariant"]["id"]
        all_prices_df.append(create_df(price_data))

        del data["selectedVariant"]["price"]
        del data["selectedVariant"]["packageInfo"]["unitPrice"]
        data["selectedVariant"]["packageUnit"] = data["selectedVariant"]["packageInfo"]["packageUnit"]
        data["selectedVariant"]["packageSize"] = data["selectedVariant"]["packageInfo"]["packageSize"]
        del data["selectedVariant"]["packageInfo"]

        all_variant_df.append(create_df(data["selectedVariant"]))

        data["selectedVariant"] = data["selectedVariant"]["id"]
        data["defaultVariant"] = data["defaultVariant"]["id"]
        all_product_df.append(create_df(data))

df_products = pd.concat(all_product_df, ignore_index=True)
df_variants = pd.concat(all_variant_df, ignore_index=True)
df_prices = pd.concat(all_prices_df, ignore_index=True)
end_time = time.time()
print(end_time - start_time)
print((end_time - start_time) / len(df_products))

684.7651336193085
0.008746185912141678


In [11]:
df_prices.head(10)

,net,gross,currency,decimalPlaces,netDiscounted,grossDiscounted,discountPercentage,isDiscounted,id,discountValidFrom,discountValidTo
0,30799,30799,HUF,0,30799,30799,0,False,4073949,NaN,NaN
1,479,479,HUF,0,479,479,0,False,39247,NaN,NaN
2,579,579,HUF,0,579,579,0,False,4041974,NaN,NaN
3,639,639,HUF,0,639,639,0,False,4040990,NaN,NaN
4,989,989,HUF,0,989,989,0,False,4019218,NaN,NaN
5,1149,1149,HUF,0,1149,1149,0,False,3613051,NaN,NaN
6,1199,1199,HUF,0,1199,1199,0,False,3677631,NaN,NaN
7,1199,1199,HUF,0,1199,1199,0,False,3613153,NaN,NaN
8,1299,1299,HUF,0,1299,1299,0,False,3612033,NaN,NaN
9,1469,1469,HUF,0,1469,1469,0,False,3952491,NaN,NaN


In [18]:
df_variants.drop_duplicates(subset=["id"], inplace=True)
df_products.drop_duplicates(subset=["id"], inplace=True)
df_prices.drop_duplicates(subset=["id"], inplace=True)
df_variants

,id,name,sku,productId,addedName,selectValue,status,unit,eanCode,aided,...,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,packageUnit,packageSize,itemVolumeInfo,roll
0,4073949,Lars etetőszék,524266,536559,,4073949,none,db,3830071133072,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.0,NaN,NaN
1,39247,Alomlapát 28cm,855966,39244,,39247,none,db,8585027970144,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.0,NaN,NaN
2,4041974,Lapát Kicsi Zárt,494729,504572,,4041974,none,db,4011905040455,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.0,NaN,NaN
3,4040990,Trixie 4046 Alomlapát Több Féle Színben,470695,503591,,4040990,none,db,4011905040462,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.0,NaN,NaN
4,4019218,"Alom illatosító, virág",452051,481822,,4019218,none,db,3166780147001,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75096,3855896,Nándi Mese kirakó,583152,318605,,3855896,none,db,5999536282019,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, parameterList]",False,[],,DB,1.0,NaN,NaN
75100,3855893,Nándi Mese Memória,583151,318602,,3855893,none,db,5999536282002,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, parameterList]",False,[],,DB,1.0,NaN,NaN
75190,4058062,Smart Sketcher pro,521140,520672,,4058062,none,db,854617005176,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, parameterList]",False,[],,DB,1.0,NaN,NaN
75621,4067029,Unmatched: Piroska vs. Beowulf társasjáték,547855,529639,,4067029,none,db,5999566844386,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, parameterList]",False,[],,DB,1.0,NaN,NaN


In [19]:
assert len(df_products) == len(df_variants) == len(df_prices)

In [20]:
df_variants.to_csv("../databases/variants_raw2.csv", escapechar="\\", compression='xz')
df_products.to_csv("../databases/products_raw2.csv", escapechar="\\", compression='xz')
df_prices.to_csv("../databases/prices_raw2.csv", escapechar="\\", compression='xz')

In [70]:
df_variants.loc[df_variants["details"].str.contains("allergens").fillna(False)]

,id,name,sku,price,productId,addedName,selectValue,status,unit,eanCode,...,packageInfo,loose,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,itemVolumeInfo,roll


In [91]:
df_variants["details"].str[2].str.contains("allergens")

0        False
1        False
2        False
3        False
4        False
         ...  
34808      NaN
34809      NaN
34810      NaN
34811      NaN
34812      NaN
Name: details, Length: 34813, dtype: object

In [100]:
df_variants.loc[df_variants["details"].str[-1] == "allergens", "id"]

0          14914
2        4001800
3        4001797
4        3988465
5        3989383
          ...   
28985    4117302
28990    4117341
28994    4117335
28996    4117305
28997    4117344
Name: id, Length: 8168, dtype: int64

In [4]:
df_variants = pd.read_csv("../databases/variants_raw.csv", index_col=0, compression='xz')
df_products = pd.read_csv("../databases/products_raw.csv", index_col=0, compression='xz')
df_prices = pd.read_csv("../databases/prices_raw.csv", index_col=0, compression='xz')

/tmp/ipykernel_6060/2871604282.py:1: DtypeWarning: Columns (5,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_variants = pd.read_csv("../variants_raw.csv", index_col=0, compression='xz')


In [20]:
df_products

,id,categoryId,categoryName,brandName,defaultVariant,selectedVariant,eancode,usedItem,reviewable,reviewSum,categories,inCategories,isNewProduct,stockInfos,adultsOnly,shipmentDays,ageConfirmed,isNonFood,documents,extraWeightPrice
0,14911,5906,Fagyasztott péksütemény,Landly,14914,14914,4050529071186,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 1, 'name': 'Élelmiszer', 'imageUrl': '...",[5906],False,[],False,0,False,False,[],NaN
1,425635,5906,Fagyasztott péksütemény,MEKKO,3963016,3963016,5999887426261,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 1, 'name': 'Élelmiszer', 'imageUrl': '...",[5906],False,[],False,0,False,False,[],NaN
2,464410,5906,Fagyasztott péksütemény,Ledo,4001800,4001800,3850116038957,"{'canReturn': False, 'return': False}",login,"{'sumCount': 1, 'average': 5}","[{'id': 1, 'name': 'Élelmiszer', 'imageUrl': '...",[5906],False,[],False,0,False,False,[],NaN
3,464407,5906,Fagyasztott péksütemény,Ledo,4001797,4001797,3850116038926,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 1, 'name': 'Élelmiszer', 'imageUrl': '...",[5906],False,[],False,0,False,False,[],NaN
4,451075,5906,Fagyasztott péksütemény,Auchan,3988465,3988465,3596710371617,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 1, 'name': 'Élelmiszer', 'imageUrl': '...",[5906],False,[],False,0,False,False,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34808,582898,13076,Szárítás,Actuel,4120288,4120288,3245678384039,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[13076],False,[],False,0,False,True,[],NaN
34809,582901,13076,Szárítás,NaN,4120291,4120291,9007741018777,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[13076],False,[],False,0,False,True,[],NaN
34810,582985,13079,"Ruháskosár, szennyestartó",Curver,4120375,4120375,3253920767011,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[13079],False,[],False,0,False,True,[],NaN
34811,582982,13079,"Ruháskosár, szennyestartó",Curver,4120372,4120372,3253920767028,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[13079],False,[],False,0,False,True,[],NaN


In [19]:
df_variants

,id,name,sku,productId,addedName,selectValue,status,unit,eanCode,aided,...,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,packageUnit,packageSize,itemVolumeInfo,roll
0,14914,"Landly gyorsfagyasztott, elősütött baguette, z...",509341,14911,,14914,none,db,4050529071186,False,...,"[{'flag': 'flag_discount', 'name': 'Kiemelt aj...",{'images': ['https://ahuazurewebblob0.azureedg...,"[ingredients, parameterList, nutrition, allerg...",False,[],discount,KG,0.175,NaN,NaN
1,3963016,"MEKKO gyorsfagyasztott, elősütött bagett, Zöld...",375657,425635,,3963016,none,db,5999887426261,False,...,"[{'flag': 'flag_discount', 'name': 'Kiemelt aj...",{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, parameterList, nutr...",False,[],discount,KG,0.150,NaN,NaN
2,4001800,Ledo gyorsfagyasztott étcsokoládé ízű-meggyes ...,413629,464410,,4001800,none,db,3850116038957,False,...,"[{'flag': 'flag_frozen', 'name': 'Fagyasztott ...",{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, parameterList, nutr...",False,[],,KG,0.500,NaN,NaN
3,4001797,Ledo gyorsfagyasztott vaníliás-erdei gyümölcsö...,413628,464407,,4001797,none,db,3850116038926,False,...,"[{'flag': 'flag_frozen', 'name': 'Fagyasztott ...",{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, parameterList, nutr...",False,[],,KG,0.500,NaN,NaN
4,3988465,Auchan Nívó Sütésre kész tiszta vajcsokoládé t...,279018,451075,,3988465,none,db,3596710371617,False,...,"[{'flag': 'flag_auchan_brand', 'name': 'Auchan...",{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, parameterList, nutr...",False,[],,KG,0.450,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34808,4120288,Actuel kihúzható szárító 20m,616669,582898,,4120288,none,db,3245678384039,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.000,NaN,NaN
34809,4120291,Kültéri szárító aluminium 50m,616670,582901,,4120291,none,db,9007741018777,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.000,NaN,NaN
34810,4120375,"Curver ""ribbon"" ruháskosár 40 l szürke",645711,582985,,4120375,none,db,3253920767011,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.000,NaN,NaN
34811,4120372,"Curver ""ribbon"" ruháskosár 40l fehér",645710,582982,,4120372,none,db,3253920767028,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,[description],False,[],,DB,1.000,NaN,NaN


In [5]:
df_nutritions = pd.read_csv("../nutritions_raw.csv", index_col=0)

In [6]:
df_nutritions

,id,Energia (kJ / kcal),Zsír (g),Telített zsírsavak (g),Szénhidrát (g),Cukrok (g),Rost (g),Fehérje (g),Só (g),Riboflavin (mg),...,Ebből koleszterin,A vitamin (µg),Tiamin (mg),Niacin (mg),Pantoténsav (mg),Klorid (mg),Cink (mg),Mangán (mg),Fluor (µg),Króm (µg)
0,14914,1199/285,8.8000,4.8000,42.0000,2.3000,0.000000,8.3000,1.4000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3963016,1091.830000/260.770000,7.670000,3.540000,45.350000,2.690000,3.840000,0.670000,3.840000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4001800,1270/303,14.0000,7.6000,39.0000,17.0000,NaN,4.1000,0.7800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4001797,1122/268,13.0000,7.0000,34.0000,14.0000,NaN,3.7000,0.7000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3988465,1755.000000/420.000000,23.400000,15.000000,42.400000,12.600000,NaN,8.100000,0.840000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10595,4117341,2030/601,23,9,67,33,NaN,"5,6","0,66",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10596,4117392,129/31,"0,4","0,1","6,1","0,2",NaN,"0,7","0,9",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10597,4117335,2079/625,28,11,61,30,NaN,"5,5","1,23",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10598,4117305,1835/437,18,"7,2","63,6",28,NaN,"5,1","0,3",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# merge variants with prices and nutritions
df_merged = df_variants.merge(df_prices, on="id")
df_merged = df_merged.merge(df_nutritions, on="id")

In [31]:
df_merged.sort_values(by="discountPercentage", ascending=False).head(100)[["name", "gross", "discountPercentage", "grossDiscounted"]]

,name,gross,discountPercentage,grossDiscounted
10417,Ischler kajsziízzel töltve,199,50,99
1351,Thymos Csillagánizs egész 5 g,335,50,169
10419,Porcukros linzer kajszibarack ízű töltelékkel ...,199,50,99
10418,Morzsás linzer kajszibarack ízű töltelékkel tö...,199,50,99
10481,St Jacques kagylóhús 250 g,7789,49,4000
...,...,...,...,...
1076,De Cecco spaghetti durum búzadarából készült t...,1249,30,874
6156,Bonduelle gyorsfagyasztott zöldségleves keveré...,475,30,333
6168,Bonduelle gyorsfagyasztott karfiolos krémleves...,580,30,406
6177,"Vici gyorsfagyasztott, előfőzött gombóc hússal...",1999,30,1399


In [11]:
df_merged.loc[df_merged["name"].str.contains("zab"), ["name", "Fehérje (g)"]]

,name,Fehérje (g)
53,Auchan Nívó Bio zabital 1 l,"0,5"
85,Delicious joghurtos-vörös áfonyás zabkása 65 g,13.0000
86,Dr. Oetker Vitalis laktózmentes vörös áfonyás...,"2,9"
87,"Cornexi almás fahéjas zabkása, hozzáadott cuko...",13.0000
88,Dr. Oetker Vitalis banános zabkása 55 g,3.3000
...,...,...
10263,BioTechUSA Oat&Fruit banán ízű zabszelet szárí...,5.2000
10389,Bio teljes kiőrlésű zabkekesz étcsokoládéalapp...,7.400000
10390,Teljes kiőrlésű zabkeksz tejcsokoládé talppal-...,"7,1"
10393,Biopont Bio apró zabpehely 300 g,13.0000
